# Test Generator (Girth)

Read in the candidates and item data and generate a randomised test from them using the
[Girth](https://eribean.github.io/girth/) package

We assume that the 1PL model is used.

$$
Pr(X=1) = \frac{exp(\theta-b)}{1 + exp(\theta-b)}
$$

The benefit of using the Girth library is that is it in addition to generating synthetic test data it can also be used to estimate the IRT parameters.

## Data Ingest

There are two files in the `data` folder that we need: `items.csv` and `candidates.csv`. If you want to generate a randomised set of candidates, then run the `generateCandidates` notebook first. Note: this will overwrite the `candidates.csv` file.

In [1]:
import numpy as np
from numpy.random import seed
from typing import List, Tuple
from csv import reader
import pandas as pd
from girth.synthetic import create_synthetic_irt_dichotomous
from girth import onepl_mml


def getDataAsList(datafile: str) -> List[Tuple]:
    """Turn a CSV datafile into a list of tuples

    :param datafile: the CSV file to load data from
    :return: a list of rows (tuples)
    """
    with open(datafile, 'r', encoding='utf-8-sig') as fs:
        csv_reader = reader(fs)
        row_list = list(map(tuple, csv_reader))
        return row_list[1:]    # ignore the header row
    

# convert the raw data into a simple duple of ( ucid, theta )
def getCandidates() -> List[Tuple]:
    candidates = getDataAsList('data/candidates.csv')
    new_list = [(c[0], float(c[1])) for c in candidates]
    return new_list
    

# convert the raw data into a simple triple of ( uiid, a, b )
def getItems() -> List[Tuple]:
    items = getDataAsList('data/items.csv')
    new_list = [(i[0], float(i[1]), float(i[2])) for i in items]
    return new_list

In [2]:
items = getItems()
candidates = getCandidates()

## Test Generation
We use the `create_synthetic_irt_dichotomous()` function from Girth to create the random test data. The `generateTest()` function reads data into the the numpy arrays that Girth requires, and then generates a randomised test before converting it into a pandas data frame for display.

In [3]:
def convertTupleListToArray(tl: List[Tuple], arity: int):
    l = [i[arity] for i in tl]
    return np.array(l)


def generateTest(itemList: List[Tuple], candidateList: List[Tuple], seed: int = None):
    discrimination = convertTupleListToArray(itemList, 1)
    difficulty = convertTupleListToArray(itemList, 2)
    theta = convertTupleListToArray(candidateList, 1)
    t = create_synthetic_irt_dichotomous(difficulty, discrimination, theta, seed=seed)
    return t


def convertTestToDataframe(test, itemList: List[Tuple], candidateList: List[Tuple]):
    header = []
    for i in items:
        header.append(i[0])
    rownames = []
    for i in candidates:
        rownames.append(i[0])
    df = pd.DataFrame(test.T, index=rownames, columns=header)
    return df



In [4]:
synthetic_test = generateTest(items, candidates)
testDf = convertTestToDataframe(synthetic_test, items, candidates)

In [5]:
(testDf)

,A1L_7616_01#6789,A1L_20679_02#6790,A1L_5480_03#6791,A2L_5483_04#6792,A2L_24442_05#6793,A2L_7620_06#6794,A2L_7627_07#6795,B1L_20849_08#6796,B1L_4287_09#6797,B1L_21135_10#6798,...,B1VG_3919_SA06,B2VG_5843_SA07,C1VG_4378_SA08,C2VG_8173_SA09,C2VG_18944_SA10,B2R_4464_WA_01,B2R_4161_WA_02,C1R_4135_WA_03,C1R_4421_WA_04,C1R_4136_WA_05
DT0001,1,1,1,1,1,1,1,1,1,1,...,0,0,1,1,0,1,0,1,1,0
DT0002,0,1,0,1,1,1,1,1,0,0,...,1,1,0,0,0,1,0,0,0,0
DT0003,1,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DT0004,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
DT0005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DT0396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DT0397,0,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
DT0398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
DT0399,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


### GenerateRandomTests function
We also include a `GenerateRandomTests()` function that is the same as the one in the `generateTest.ipynb` notebook. You can then call the `GenerateRandomTests()` function as many times as you want to re-generate a test. It will generate different results every time (unless you pass in an integer seed value).

Add items and candidates to the data files to generate larger tests.

When you are happy with the results you can write out to a results CSV file.

In [6]:
def GenerateRandomTests(seed: int = None):
    synthetic_test = generateTest(items, candidates, seed)
    testDf = convertTestToDataframe(synthetic_test, items, candidates)
    return testDf

In [7]:
df = GenerateRandomTests(89)

(df)

,A1L_7616_01#6789,A1L_20679_02#6790,A1L_5480_03#6791,A2L_5483_04#6792,A2L_24442_05#6793,A2L_7620_06#6794,A2L_7627_07#6795,B1L_20849_08#6796,B1L_4287_09#6797,B1L_21135_10#6798,...,B1VG_3919_SA06,B2VG_5843_SA07,C1VG_4378_SA08,C2VG_8173_SA09,C2VG_18944_SA10,B2R_4464_WA_01,B2R_4161_WA_02,C1R_4135_WA_03,C1R_4421_WA_04,C1R_4136_WA_05
DT0001,1,1,1,1,1,1,1,0,1,1,...,0,1,1,0,0,1,1,1,1,0
DT0002,0,1,0,0,1,1,1,0,0,0,...,1,1,0,0,1,1,1,1,1,0
DT0003,0,0,1,0,1,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
DT0004,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
DT0005,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DT0396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DT0397,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
DT0398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DT0399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df.to_csv('data/results.csv', index=True)

### Solving using Standard Estimation
You can use either maximum marginal likelihood (MML) or joint maximum likelihood (JML) estimation methods with the Girth library. Here we use MML estimation for the 1PL model (`onepl_mml`) to separately estimate the item parameters (`a` is the discrimination parameter and `bs` is an array of item difficulties) using univariate optimization methods. 

In [9]:
estimates = onepl_mml(synthetic_test)

a = estimates['Discrimination']
bs = estimates['Difficulty']

In [10]:
(a)

2.058668203464576

In [11]:
(bs)

array([ 0.19663654,  0.05714915,  0.05714915,  0.33008294,  0.27134397,
        0.43248719,  0.34698815,  0.44113666,  0.52886078,  0.70309066,
        0.57364336,  0.64680706,  0.85032268,  0.96572097,  1.10129065,
        1.1250714 ,  0.43248719,  0.42385718,  0.38098298,  0.35546318,
        0.38952254,  0.22145244,  0.38952254,  0.48471063,  0.33008294,
        0.29644077,  0.38098298,  0.42385718,  0.31323487,  0.36395372,
        0.25467122,  0.46724648,  0.28806297,  0.54669396,  0.2796975 ,
        0.32165196,  0.40665369,  0.37246018,  0.36395372,  0.40665369,
        0.38098298,  0.65609459,  0.45852647,  0.27134397,  0.44982123,
        0.3385282 ,  0.32165196,  0.33008294,  0.30483127,  0.34698815,
       -0.12261377,  0.08165808,  0.14723492,  0.22145244,  0.33008294,
        0.32165196,  0.52886078,  0.38098298,  0.54669396,  0.55564973,
        0.33008294,  0.33008294,  0.44982123,  0.46724648,  0.47595076,
        0.40665369,  0.40665369,  0.30483127,  0.40665369,  0.42